# Chapter 5: Brute-Force Algorithm Validation

## Introduction
In this chapter, we will subject our signature algorithm to a rigorous validation test. The objective is to verify if it can correctly discriminate between all graph structures for a given order `n` by checking it against established mathematical results.

### The Fundamental Hypothesis

A canonical signature algorithm is correct if and only if it meets two fundamental conditions:
1.  **Consistency**: It must assign the same signature to two graphs that are structurally identical (*isomorphic*).
2.  **Uniqueness**: It must assign different signatures to two graphs that are structurally different (*non-isomorphic*).

### Testing Methodology

Rather than starting with a pre-filtered list of unique graphs, we will adopt a "brute-force" approach that tests both conditions simultaneously. This method is more comprehensive as it makes no initial assumptions about the graphs.

The process is as follows:
1.  For a given order `n`, we determine the number of possible edges, `k = n * (n - 1) / 2`.
2.  We generate all **`2^k`** possible labeled graphs. This set contains every possible combination of edges between the `n` vertices.
3.  For each of these graphs, we calculate its canonical signature using our algorithm.
4.  We use a `set` to count the number of unique signatures obtained at the end of the process.

### Validation Criterion

The test is considered a **success** if the number of unique signatures we discover is exactly equal to the known number of non-isomorphic graphs for order `n`. This reference value is provided by the **OEIS A000088** sequence (On-Line Encyclopedia of Integer Sequences).

### Scope of the Experiment and Computational Cost

This method is exhaustive, but its computational cost increases exponentially. We will therefore begin by validating our algorithm for orders `n` from 3 to 6. The table below illustrates the explosion in the number of cases to be processed.

| Order (n) | Possible Edges (k) | Graphs to Test (2^k)  | Expected Unique Graphs (OEIS A000088) |
| :-------- | :------------------- | :---------------------- | :-------------------------------------- |
| 3         | 3                    | 8                       | 4                                       |
| 4         | 6                    | 64                      | 11                                      |
| 5         | 10                   | 1,024                   | 34                                      |
| 6         | 15                   | 32,768                  | 156                                     |
| 7* | 21                   | 2,097,152               | 1,044                                   |
| 8* | 28                   | 268,435,456             | 12,005                                  |
| 9* | 36                   | 68,719,476,736          | 274,668                                 |
| 10*| 45                   | ~3.5 x 10^13            | 12,005,168                              |

*Orders marked with an asterisk require considerable processing time.*



## 5.1 Order 3

In [1]:
from collections import defaultdict
from utils import get_signature, check

# The 8 possible labeled graphs of order 3
graphs_order_3_g6 = [
    'B_',  # 0 edges (1 graph)
    'BQ', 'BS', 'Bw',  # 1 edge (3 graphs)
    'BR', 'Bi', 'Bq',  # 2 edges (3 graphs)
    'B~'   # 3 edges (1 graph)
]

# Dictionary to store the results: {signature: [list_of_g6_strings]}
signature_groups = defaultdict(list)

print(f"Processing {len(graphs_order_3_g6)} graphs of order 3...")

# Loop through all 8 graphs
for g6 in graphs_order_3_g6:
    gs = get_signature(g6)
    compact_signature = gs.sig()
    signature_groups[compact_signature].append(g6)
    print(f"g6:{g6} sig:{compact_signature}")

# --- Analysis of Results ---

num_unique_found = len(signature_groups)
num_expected = 4  # From OEIS A000088 for n=3

print("--- Results for n=3 ---")

check(
    f"We expect {num_expected} unique signatures and got {num_unique_found}", num_unique_found == num_expected)

Processing 8 graphs of order 3...
g6:B_ sig:[{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:BQ sig:[{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:BS sig:[{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bw sig:[{nc:2,n:[{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]},{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]},{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]},{nc:2,n:[{nc:2,rs:3,ll:2},{nc:2,n:[{nc:2,rs:4,ll:2},{nc:2,rs:4,ll:3}]}]}]}]
g6:BR sig:[{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,rs:3,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bi sig:[{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]}